In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

Получение целевой метрики по кросс-валидации c **тремя** фолдами и метрикой **roc_auc** LogisticRegression

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

# verbose отвечает за детализацию выводимой информации. verbose=2 значит, что будет выводится score, а именно roc-auc-score

cross_val_score(LogisticRegression(), x, y, cv=3, scoring='roc_auc', verbose=2).mean()

[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


0.8250515880497281

Подбор гиперпараметров по сетке **(GridSearchCV)**

In [11]:
from sklearn.model_selection import GridSearchCV

params = {'C' : np.arange(0.01, 10.01, 0.25)} 

gs = GridSearchCV(LogisticRegression(), params, cv=3, scoring='roc_auc')
# первый параметр - модель, второй - словарь параметров, третий - число фолдов, четвертый - метрика

gs.fit(x, y) # применяем метод к данным X, y (GridSearchCV сам разобъет данные на фолды)

print(gs.best_score_) # качество наилучшей модели (с оптимальными гиперпараметрами)
print(gs.best_params_) # оптимальные гиперпараметры

0.825109999279449
{'C': 0.26}


In [88]:
# перебор гиперпараметров из списка
params2 = {'kernel' : ['rbf', 'poly', 'sigmoid', 'linear']}

gs2 = GridSearchCV(SVC(), params2, cv=3, scoring='roc_auc')

gs2.fit(x, y)

print(gs2.best_score_) # качество наилучшей модели (с оптимальными гиперпараметрами)
print(gs2.best_params_) # оптимальные гиперпараметры

0.8057287165669725
{'kernel': 'rbf'}


gs2.best_estimator_    # Это уже **обученная модель с наилучшими параметрами**. Ее можно использовать для получения предсказания

### GridsearchCV с пайплайном

Код из примера

In [ ]:
pipe = Pipeline(
    [
        ("imputer", SimpleImputer()),
        ("scaler", StandardScaler()),
        ("regressor", RandomForestRegressor()),
    ]
)

param_grid = [
    {
        "imputer__strategy": ["mean", "median", "most_frequent", "constant"],
        "scaler": [StandardScaler(), MinMaxScaler(), "passthrough"],
        "regressor": [RandomForestRegressor(random_state=42)],
        "regressor__n_estimators": [100, 300, 500, 1000],
        "regressor__max_depth": [3, 5, 8, 15],
        "regressor__max_features": ["log2", "sqrt", "auto"],
    },
    {
        "regressor": [XGBRegressor(random_state=42)],
        "regressor__max_depth": [3, 5, 8, 15],
        "regressor__learning_rate": [0.1, 0.01, 0.05],
        "regressor__gamma": [0, 0.25, 1.0],
        "regressor__lambda": [0, 1.0, 10.0],
    },
]

grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    return_train_score=True,
)

Реально выполненный **рабочий пример**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV

p1 = Pipeline([
    ('features', PolynomialFeatures(degree=2)),
    ('model', DecisionTreeRegressor(max_depth=3, random_state=42))
])

params = {'model__max_depth' : np.arange(3, 11, 1)} 

gs2 = GridSearchCV(p1, params, cv=3, scoring='r2')

gs2.fit(X_train, y_train)

print(gs2.best_score_)
print(gs2.best_params_)
model = gs2.best_estimator_